In [3]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path

def load_metrics_for_comparison(platform, metric_name, scenario):
    """
    Load metrics from the platform's output directory for comparison
    
    Args:
        platform: Name of the platform (JADE, SPADE)
        metric_name: Name of the metric (capacity, compactness, occupancy, time_eligibility, RE)
        scenario: Scenario name (full, medium, small)
        
    Returns:
        Loaded metrics data or None if not found
    """
    try:
        # Construct the path to the metrics file
        if metric_name == 'RE':
            # RE metrics are stored differently
            file_path = os.path.join(
                "../scheduling_output", 
                platform, 
                metric_name, 
                "all", 
                f"{platform}_RE_analysis.json"
            )
            
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                # Extract scenario-specific data
                return data['scenarios'][scenario]
        else:
            # Regular metrics stored in scenario-specific folders
            metric_filename = {
                'capacity': 'metricas_capacidad.json',
                'compactness': 'metricas_compactacion.json',
                'occupancy': 'metricas_ocupacion.json',
                'time_eligibility': 'metricas_te.json'
            }
            
            file_path = os.path.join(
                "../scheduling_output", 
                platform, 
                metric_name, 
                scenario, 
                metric_filename[metric_name]
            )
            
            with open(file_path, 'r', encoding='utf-8') as f:
                return json.load(f)
                
    except (FileNotFoundError, json.JSONDecodeError, KeyError) as e:
        print(f"Error loading {metric_name} metrics for {platform}, {scenario}: {str(e)}")
        return None

def create_comparison_dataframe(metric_name, platforms=['JADE', 'SPADE'], scenarios=['full', 'medium', 'small']):
    """
    Creates a DataFrame for cross-platform comparison of a specific metric
    
    Args:
        metric_name: The metric to compare
        platforms: List of platforms to compare
        scenarios: List of scenarios to include
        
    Returns:
        DataFrame with comparison data
    """
    comparison_data = []
    
    for scenario in scenarios:
        row = {'Scenario': scenario}
        
        for platform in platforms:
            # Load metrics for this platform and scenario
            metrics = load_metrics_for_comparison(platform, metric_name, scenario)
            
            if metrics is None:
                continue
                
            # Extract relevant metrics based on metric type
            if metric_name == 'capacity':
                # For capacity metrics, get the global stats
                row[f"{platform}_bajo_capacidad"] = metrics['global_stats']['bajo_capacidad']
                row[f"{platform}_capacidad_exacta"] = metrics['global_stats']['capacidad_exacta']
                row[f"{platform}_sobre_capacidad"] = metrics['global_stats']['sobre_capacidad']
                
            elif metric_name == 'compactness':
                # For compactness, calculate average windows per room
                total_ventanas = sum(room['ventanas'] for room in metrics['estadisticas_por_sala'].values())
                total_rooms = len(metrics['estadisticas_por_sala'])
                row[f"{platform}_avg_ventanas"] = total_ventanas / total_rooms if total_rooms > 0 else 0
                
            elif metric_name == 'occupancy':
                # For occupancy, get the global occupancy rate
                row[f"{platform}_tasa_ocupacion"] = metrics['global_stats']['tasa_ocupacion_global']
                
            elif metric_name == 'time_eligibility':
                # For time eligibility, get the TE percentage
                row[f"{platform}_te_percentage"] = metrics['te_percentage']
                
            elif metric_name == 'RE':
                # For RE metrics, get the RE percentage and related metrics
                row[f"{platform}_re_percentage"] = metrics['re_percentage']
                if 'schedule_quality' in metrics and metrics['schedule_quality']:
                    row[f"{platform}_scheduling_rate"] = metrics['schedule_quality'].get('scheduling_rate_percentage', 0)
                    row[f"{platform}_avg_satisfaction"] = metrics['schedule_quality'].get('avg_satisfaction', 0)
        
        comparison_data.append(row)
    
    # Create DataFrame
    return pd.DataFrame(comparison_data)

def create_comparative_charts(metric_name, df, output_dir="../scheduling_output/comparative"):
    """
    Creates comparative charts for the specified metric
    
    Args:
        metric_name: The metric being compared
        df: DataFrame with comparison data
        output_dir: Directory to save output charts
    """
    # Create output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    if metric_name == 'capacity':
        # Create grouped bar chart for capacity metrics
        fig, ax = plt.subplots(figsize=(12, 8))
        
        x = np.arange(len(df['Scenario']))  # the label locations
        width = 0.15  # the width of the bars
        
        # Create bars for each platform and capacity type
        platforms = ['JADE', 'SPADE']
        metrics = ['bajo_capacidad', 'capacidad_exacta', 'sobre_capacidad']
        colors = ['#99C2E8', '#00629B', '#A9A9A9']
        
        for i, platform in enumerate(platforms):
            for j, metric in enumerate(metrics):
                col = f"{platform}_{metric}"
                if col in df.columns:
                    offset = width * (i * len(metrics) + j - len(metrics))
                    ax.bar(x + offset, df[col], width, label=f"{platform} - {metric.replace('_', ' ').title()}", 
                           color=colors[j], alpha=0.7 if platform == 'SPADE' else 1.0)
        
        # Add labels and legend
        ax.set_ylabel('Percentage (%)')
        ax.set_title('Capacity Distribution Comparison: JADE vs SPADE')
        ax.set_xticks(x)
        ax.set_xticklabels(df['Scenario'])
        ax.legend()
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'capacity_comparison.png'))
        plt.close()
        
    elif metric_name == 'compactness':
        # Create bar chart for average windows per room
        plt.figure(figsize=(10, 6))
        
        jade_data = df['JADE_avg_ventanas'].values
        spade_data = df['SPADE_avg_ventanas'].values
        
        x = np.arange(len(df['Scenario']))
        width = 0.35
        
        plt.bar(x - width/2, jade_data, width, label='JADE', color='#00629B')
        plt.bar(x + width/2, spade_data, width, label='SPADE', color='#A9A9A9')
        
        plt.xlabel('Scenario')
        plt.ylabel('Average Windows per Room')
        plt.title('Compactness Comparison: JADE vs SPADE')
        plt.xticks(x, df['Scenario'])
        plt.legend()
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'compactness_comparison.png'))
        plt.close()
        
    elif metric_name == 'occupancy':
        # Create bar chart for occupancy rate
        plt.figure(figsize=(10, 6))
        
        jade_data = df['JADE_tasa_ocupacion'].values
        spade_data = df['SPADE_tasa_ocupacion'].values
        
        x = np.arange(len(df['Scenario']))
        width = 0.35
        
        plt.bar(x - width/2, jade_data, width, label='JADE', color='#00629B')
        plt.bar(x + width/2, spade_data, width, label='SPADE', color='#A9A9A9')
        
        plt.xlabel('Scenario')
        plt.ylabel('Occupancy Rate (%)')
        plt.title('Room Occupancy Comparison: JADE vs SPADE')
        plt.xticks(x, df['Scenario'])
        plt.legend()
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'occupancy_comparison.png'))
        plt.close()
        
    elif metric_name == 'time_eligibility':
        # Create bar chart for time eligibility
        plt.figure(figsize=(10, 6))
        
        jade_data = df['JADE_te_percentage'].values
        spade_data = df['SPADE_te_percentage'].values
        
        x = np.arange(len(df['Scenario']))
        width = 0.35
        
        plt.bar(x - width/2, jade_data, width, label='JADE', color='#00629B')
        plt.bar(x + width/2, spade_data, width, label='SPADE', color='#A9A9A9')
        
        plt.xlabel('Scenario')
        plt.ylabel('Time-slot Eligibility (%)')
        plt.title('Time-slot Eligibility Comparison: JADE vs SPADE')
        plt.xticks(x, df['Scenario'])
        plt.legend()
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'time_eligibility_comparison.png'))
        plt.close()
        
    elif metric_name == 'RE':
        # Create comparison charts for Room Eligibility
        fig, axs = plt.subplots(1, 2, figsize=(15, 6))
        
        # 1. RE percentage chart
        x = np.arange(len(df['Scenario']))
        width = 0.35
        
        axs[0].bar(x - width/2, df['JADE_re_percentage'].values, width, label='JADE', color='#00629B')
        axs[0].bar(x + width/2, df['SPADE_re_percentage'].values, width, label='SPADE', color='#A9A9A9')
        
        axs[0].set_xlabel('Scenario')
        axs[0].set_ylabel('Room Eligibility (%)')
        axs[0].set_title('Room Eligibility Comparison')
        axs[0].set_xticks(x)
        axs[0].set_xticklabels(df['Scenario'])
        axs[0].legend()
        
        # 2. Satisfaction score chart
        if 'JADE_avg_satisfaction' in df.columns and 'SPADE_avg_satisfaction' in df.columns:
            axs[1].bar(x - width/2, df['JADE_avg_satisfaction'].values, width, label='JADE', color='#00629B')
            axs[1].bar(x + width/2, df['SPADE_avg_satisfaction'].values, width, label='SPADE', color='#A9A9A9')
            
            axs[1].set_xlabel('Scenario')
            axs[1].set_ylabel('Average Satisfaction Score')
            axs[1].set_title('Schedule Satisfaction Comparison')
            axs[1].set_xticks(x)
            axs[1].set_xticklabels(df['Scenario'])
            axs[1].legend()
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 're_comparison.png'))
        plt.close()

def create_comparative_summary_table(platforms=['JADE', 'SPADE'], scenarios=['full', 'medium', 'small']):
    """Creates a comprehensive summary table comparing all metrics across platforms"""
    
    # Dictionary to store all metrics for each platform and scenario
    all_metrics = {}
    
    # Metrics to collect
    metrics = ['capacity', 'compactness', 'occupancy', 'time_eligibility', 'RE']
    
    # Gather all metrics
    for platform in platforms:
        all_metrics[platform] = {}
        
        for scenario in scenarios:
            all_metrics[platform][scenario] = {}
            
            for metric in metrics:
                data = load_metrics_for_comparison(platform, metric, scenario)
                if data:
                    all_metrics[platform][scenario][metric] = data
    
    # Build comparison data for table
    comparison_rows = []
    
    for scenario in scenarios:
        for platform in platforms:
            row = {
                'Platform': platform,
                'Scenario': scenario
            }
            
            # Add capacity metrics
            if 'capacity' in all_metrics[platform][scenario]:
                cap_data = all_metrics[platform][scenario]['capacity']
                row['Optimal Capacity (%)'] = cap_data['global_stats']['capacidad_exacta']
                row['Under Capacity (%)'] = cap_data['global_stats']['bajo_capacidad']
            
            # Add occupancy metrics
            if 'occupancy' in all_metrics[platform][scenario]:
                occ_data = all_metrics[platform][scenario]['occupancy']
                row['Room Occupancy (%)'] = occ_data['global_stats']['tasa_ocupacion_global']
            
            # Add compactness metrics (average windows)
            if 'compactness' in all_metrics[platform][scenario]:
                comp_data = all_metrics[platform][scenario]['compactness']
                total_ventanas = sum(room['ventanas'] for room in comp_data['estadisticas_por_sala'].values())
                total_rooms = len(comp_data['estadisticas_por_sala'])
                row['Avg Windows/Room'] = total_ventanas / total_rooms if total_rooms > 0 else 0
            
            # Add time eligibility
            if 'time_eligibility' in all_metrics[platform][scenario]:
                te_data = all_metrics[platform][scenario]['time_eligibility']
                row['TE (%)'] = te_data['te_percentage']
            
            # Add RE metrics
            if 'RE' in all_metrics[platform][scenario]:
                re_data = all_metrics[platform][scenario]['RE']
                row['RE (%)'] = re_data['re_percentage']
                if 'schedule_quality' in re_data and re_data['schedule_quality']:
                    row['Scheduling Rate (%)'] = re_data['schedule_quality'].get('scheduling_rate_percentage', 0)
                    row['Satisfaction Score'] = re_data['schedule_quality'].get('avg_satisfaction', 0)
            
            comparison_rows.append(row)
    
    # Create DataFrame and save to CSV
    df = pd.DataFrame(comparison_rows)
    
    # Save the table
    output_dir = "../scheduling_output/comparative"
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    df.to_csv(os.path.join(output_dir, 'platform_comparison_summary.csv'), index=False)
    
    return df

def generate_all_comparisons():
    """Generates all comparison charts and tables"""
    output_dir = "../scheduling_output/comparative"
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Generate comparison for each metric
    metrics = ['capacity', 'compactness', 'occupancy', 'time_eligibility', 'RE']
    
    for metric in metrics:
        print(f"Generating comparison for {metric}...")
        df = create_comparison_dataframe(metric)
        df.to_csv(os.path.join(output_dir, f'{metric}_comparison.csv'), index=False)
        create_comparative_charts(metric, df, output_dir)
    
    # Generate comprehensive summary table
    print("Generating comprehensive comparison summary...")
    summary_df = create_comparative_summary_table()
    
    # Create summary visualization
    create_radar_chart_comparison(summary_df, output_dir)
    
    print(f"All comparisons generated and saved to {output_dir}")
    return summary_df

def create_radar_chart_comparison(df, output_dir):
    """Creates a radar chart comparing overall performance across metrics"""
    
    # We'll create one radar chart per scenario
    scenarios = df['Scenario'].unique()
    
    for scenario in scenarios:
        # Filter data for this scenario
        scenario_data = df[df['Scenario'] == scenario]
        
        # Metrics to use in radar chart
        metrics = [
            'Optimal Capacity (%)', 
            'Room Occupancy (%)',
            'Avg Windows/Room',
            'TE (%)',
            'RE (%)',
            'Satisfaction Score'
        ]
        
        # Scale and normalize the data for radar chart
        max_values = {}
        for metric in metrics:
            if metric in scenario_data.columns:
                max_val = scenario_data[metric].max()
                if metric == 'Avg Windows/Room':
                    # For windows, lower is better, so we invert the scale
                    max_values[metric] = max_val
                else:
                    max_values[metric] = max_val
        
        # Normalize the metrics (scale 0-1)
        normalized_data = {}
        for platform in ['JADE', 'SPADE']:
            platform_data = scenario_data[scenario_data['Platform'] == platform]
            if len(platform_data) == 0:
                continue
                
            normalized_data[platform] = []
            for metric in metrics:
                if metric in platform_data.columns and metric in max_values and max_values[metric] > 0:
                    value = platform_data[metric].values[0]
                    if metric == 'Avg Windows/Room':
                        # Invert for windows (lower is better)
                        norm_value = 1 - (value / max_values[metric]) if value > 0 else 1
                    else:
                        # Scale for other metrics (higher is better)
                        norm_value = value / max_values[metric]
                        
                    # Cap at 0-1 range
                    norm_value = max(0, min(1, norm_value))
                    normalized_data[platform].append(norm_value)
                else:
                    normalized_data[platform].append(0)
        
        # Create the radar chart
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, polar=True)
        
        # Set number of metrics as angles
        angles = np.linspace(0, 2*np.pi, len(metrics), endpoint=False).tolist()
        angles += angles[:1]  # Close the loop
        
        # Plot platforms
        colors = {'JADE': '#00629B', 'SPADE': '#A9A9A9'}
        
        for platform, values in normalized_data.items():
            values += values[:1]  # Close the loop
            ax.plot(angles, values, 'o-', linewidth=2, color=colors[platform], label=platform)
            ax.fill(angles, values, alpha=0.25, color=colors[platform])
        
        # Set metric labels
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(metrics)
        
        # Add legend and title
        plt.legend(loc='upper right')
        plt.title(f'Timetabling Performance Comparison: {scenario.capitalize()} Scenario')
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'radar_comparison_{scenario}.png'))
        plt.close()
        
        # Also create a direct comparison bar chart
        create_head_to_head_comparison(scenario_data, metrics, scenario, output_dir)

def create_head_to_head_comparison(scenario_data, metrics, scenario, output_dir):
    """Creates a direct head-to-head bar chart comparison"""
    
    # Prepare data for bar chart
    comparison_data = []
    
    for metric in metrics:
        if metric not in scenario_data.columns:
            continue
            
        row = {'Metric': metric}
        
        for platform in ['JADE', 'SPADE']:
            platform_data = scenario_data[scenario_data['Platform'] == platform]
            if len(platform_data) > 0 and metric in platform_data.columns:
                row[platform] = platform_data[metric].values[0]
        
        comparison_data.append(row)
    
    # Create DataFrame
    comp_df = pd.DataFrame(comparison_data)
    
    # Create bar chart
    plt.figure(figsize=(12, 8))
    
    metrics = comp_df['Metric'].values
    x = np.arange(len(metrics))
    width = 0.35
    
    plt.bar(x - width/2, comp_df['JADE'].values, width, label='JADE', color='#00629B')
    plt.bar(x + width/2, comp_df['SPADE'].values, width, label='SPADE', color='#A9A9A9')
    
    plt.xlabel('Metric')
    plt.ylabel('Value')
    plt.title(f'JADE vs SPADE: {scenario.capitalize()} Scenario')
    plt.xticks(x, metrics, rotation=45, ha='right')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'head_to_head_{scenario}.png'))
    plt.close()

# If run as main script
if __name__ == "__main__":
    # Generate all comparisons
    summary_df = generate_all_comparisons()
    print("Comparison complete. Summary of results:")
    print(summary_df)

Generating comparison for capacity...
Generating comparison for compactness...
Generating comparison for occupancy...
Generating comparison for time_eligibility...
Generating comparison for RE...
Generating comprehensive comparison summary...
All comparisons generated and saved to ../scheduling_output/comparative
Comparison complete. Summary of results:
  Platform Scenario  Optimal Capacity (%)  Under Capacity (%)  \
0     JADE     full             33.592611           66.407389   
1    SPADE     full             32.714844           67.285156   
2     JADE   medium             29.248198           70.751802   
3    SPADE   medium             31.584362           68.415638   
4     JADE    small             34.876543           65.123457   
5    SPADE    small             34.876543           65.123457   

   Room Occupancy (%)  Avg Windows/Room     TE (%)     RE (%)  \
0           62.617960          0.630137  90.211754  43.518242   
1           62.343988          0.520548  89.726611  43.518

In [4]:
import json
import os
import pandas as pd
from IPython.display import display, HTML, Markdown
import numpy as np

def format_percentage(value):
    """Format a value as a percentage with 2 decimal places"""
    if isinstance(value, (int, float)):
        return f"{value:.2f}%"
    return value

def load_metrics_data(platform, metric_name, scenario):
    """
    Load metrics from the platform's output directory
    
    Args:
        platform: Name of the platform (JADE, SPADE)
        metric_name: Name of the metric (capacity, compactness, occupancy, time_eligibility, RE)
        scenario: Scenario name (full, medium, small)
        
    Returns:
        Loaded metrics data or None if not found
    """
    try:
        # Construct the path to the metrics file
        if metric_name == 'RE':
            # RE metrics are stored differently
            file_path = os.path.join(
                "../scheduling_output", 
                platform, 
                metric_name, 
                "all", 
                f"{platform}_RE_analysis.json"
            )
            
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                # Extract scenario-specific data
                return data['scenarios'][scenario]
        else:
            # Regular metrics stored in scenario-specific folders
            metric_filename = {
                'capacity': 'metricas_capacidad.json',
                'compactness': 'metricas_compactacion.json',
                'occupancy': 'metricas_ocupacion.json',
                'time_eligibility': 'metricas_te.json'
            }
            
            file_path = os.path.join(
                "../scheduling_output", 
                platform, 
                metric_name, 
                scenario, 
                metric_filename[metric_name]
            )
            
            with open(file_path, 'r', encoding='utf-8') as f:
                return json.load(f)
                
    except (FileNotFoundError, json.JSONDecodeError, KeyError) as e:
        print(f"Error loading {metric_name} metrics for {platform}, {scenario}: {str(e)}")
        return None

def print_occupancy_table():
    """Print a formatted table of occupancy metrics for all scenarios"""
    
    # Define scenarios and translations
    scenarios = {
        'small': 'Escenario pequeño', 
        'medium': 'Escenario mediano', 
        'full': 'Escenario grande'
    }
    
    print("## OCUPACION / AFORO\n")
    
    # Create a table for each scenario
    for scenario_key, scenario_name in scenarios.items():
        # Get metrics for both platforms
        jade_metrics = load_metrics_data('JADE', 'occupancy', scenario_key)
        spade_metrics = load_metrics_data('SPADE', 'occupancy', scenario_key)
        
        if not jade_metrics or not spade_metrics:
            print(f"Missing data for {scenario_name}")
            continue
        
        # Create DataFrame for this scenario
        data = {
            'METRICA': ['BLOQUES OCUPADOS', 'BLOQUES DESOCUPADOS', 'TASA DE OCUPACION'],
            'JADE': [
                jade_metrics['global_stats']['total_ocupados'],
                jade_metrics['global_stats']['total_desocupados'],
                format_percentage(jade_metrics['global_stats']['tasa_ocupacion_global'])
            ],
            'SPADE': [
                spade_metrics['global_stats']['total_ocupados'],
                spade_metrics['global_stats']['total_desocupados'],
                format_percentage(spade_metrics['global_stats']['tasa_ocupacion_global'])
            ]
        }
        df = pd.DataFrame(data)
        
        # Print scenario name and table
        print(f"### {scenario_name}:")
        display(df)
        print("\n")

def print_capacity_table():
    """Print a formatted table of capacity metrics for all scenarios"""
    
    # Define scenarios and translations
    scenarios = {
        'small': 'Escenario pequeño', 
        'medium': 'Escenario mediano', 
        'full': 'Escenario grande'
    }
    
    print("## CAPACIDAD\n")
    
    # Create a table for each scenario
    for scenario_key, scenario_name in scenarios.items():
        # Get metrics for both platforms
        jade_metrics = load_metrics_data('JADE', 'capacity', scenario_key)
        spade_metrics = load_metrics_data('SPADE', 'capacity', scenario_key)
        
        if not jade_metrics or not spade_metrics:
            print(f"Missing data for {scenario_name}")
            continue
        
        # Create DataFrame for this scenario
        data = {
            'METRICA': ['BAJO CAPACIDAD', 'CAPACIDAD EXACTA', 'SOBRE CAPACIDAD'],
            'JADE': [
                format_percentage(jade_metrics['global_stats']['bajo_capacidad']),
                format_percentage(jade_metrics['global_stats']['capacidad_exacta']),
                format_percentage(jade_metrics['global_stats']['sobre_capacidad'])
            ],
            'SPADE': [
                format_percentage(spade_metrics['global_stats']['bajo_capacidad']),
                format_percentage(spade_metrics['global_stats']['capacidad_exacta']),
                format_percentage(spade_metrics['global_stats']['sobre_capacidad'])
            ]
        }
        df = pd.DataFrame(data)
        
        # Print scenario name and table
        print(f"### {scenario_name}:")
        display(df)
        print("\n")

def print_te_table():
    """Print a formatted table of Time Eligibility metrics for all scenarios"""
    
    # Define scenarios and translations
    scenarios = {
        'small': 'Escenario pequeño', 
        'medium': 'Escenario mediano', 
        'full': 'Escenario grande'
    }
    
    print("## TIME ELIGIBILITY (TE)\n")
    
    # Create a table for each scenario
    for scenario_key, scenario_name in scenarios.items():
        # Get metrics for both platforms
        jade_metrics = load_metrics_data('JADE', 'time_eligibility', scenario_key)
        spade_metrics = load_metrics_data('SPADE', 'time_eligibility', scenario_key)
        
        if not jade_metrics or not spade_metrics:
            print(f"Missing data for {scenario_name}")
            continue
        
        # Create DataFrame for this scenario
        data = {
            'METRICA': ['TE VALUE', 'TE PERCENTAGE'],
            'JADE': [
                format_percentage(jade_metrics['te_value'] * 100),
                format_percentage(jade_metrics['te_percentage'])
            ],
            'SPADE': [
                format_percentage(spade_metrics['te_value'] * 100),
                format_percentage(spade_metrics['te_percentage'])
            ]
        }
        df = pd.DataFrame(data)
        
        # Print scenario name and table
        print(f"### {scenario_name}:")
        display(df)
        print("\n")

def print_re_table():
    """Print a formatted table of Room Eligibility metrics for all scenarios"""
    
    # Define scenarios and translations
    scenarios = {
        'small': 'Escenario pequeño', 
        'medium': 'Escenario mediano', 
        'full': 'Escenario grande'
    }
    
    print("## ROOM ELIGIBILITY (RE)\n")
    
    # Create a table for each scenario
    for scenario_key, scenario_name in scenarios.items():
        # Get metrics for both platforms
        jade_metrics = load_metrics_data('JADE', 'RE', scenario_key)
        spade_metrics = load_metrics_data('SPADE', 'RE', scenario_key)
        
        if not jade_metrics or not spade_metrics:
            print(f"Missing data for {scenario_name}")
            continue
        
        # Create DataFrame for this scenario
        data = {
            'METRICA': ['RE VALUE', 'RE PERCENTAGE', 'SATISFACTION SCORE'],
            'JADE': [
                format_percentage(jade_metrics['re_metric'] * 100),
                format_percentage(jade_metrics['re_percentage']),
                jade_metrics.get('schedule_quality', {}).get('avg_satisfaction', 'N/A')
            ],
            'SPADE': [
                format_percentage(spade_metrics['re_metric'] * 100),
                format_percentage(spade_metrics['re_percentage']),
                spade_metrics.get('schedule_quality', {}).get('avg_satisfaction', 'N/A')
            ]
        }
        df = pd.DataFrame(data)
        
        # Print scenario name and table
        print(f"### {scenario_name}:")
        display(df)
        print("\n")

def calculate_compactness_metrics(platform, scenario):
    """Calculate average windows per room and per day metrics"""
    metrics = load_metrics_data(platform, 'compactness', scenario)
    
    if not metrics:
        return {'avg_windows_per_room': 0, 'total_windows': 0, 'total_rooms': 0}
    
    # Calculate total windows
    total_windows = sum(room['ventanas'] for room in metrics['estadisticas_por_sala'].values())
    total_rooms = len(metrics['estadisticas_por_sala'])
    
    return {
        'avg_windows_per_room': total_windows / total_rooms if total_rooms > 0 else 0,
        'total_windows': total_windows,
        'total_rooms': total_rooms
    }

def print_compactness_table():
    """Print a formatted table of compactness metrics for all scenarios"""
    
    # Define scenarios and translations
    scenarios = {
        'small': 'Escenario pequeño', 
        'medium': 'Escenario mediano', 
        'full': 'Escenario grande'
    }
    
    print("## COMPACTACIÓN\n")
    
    # Create a table for each scenario
    for scenario_key, scenario_name in scenarios.items():
        # Calculate metrics for both platforms
        jade_metrics = calculate_compactness_metrics('JADE', scenario_key)
        spade_metrics = calculate_compactness_metrics('SPADE', scenario_key)
        
        # Create DataFrame for this scenario
        data = {
            'METRICA': ['VENTANAS TOTALES', 'PROMEDIO VENTANAS POR SALA'],
            'JADE': [
                jade_metrics['total_windows'],
                f"{jade_metrics['avg_windows_per_room']:.2f}"
            ],
            'SPADE': [
                spade_metrics['total_windows'],
                f"{spade_metrics['avg_windows_per_room']:.2f}"
            ]
        }
        df = pd.DataFrame(data)
        
        # Print scenario name and table
        print(f"### {scenario_name}:")
        display(df)
        print("\n")

def print_general_metrics_table():
    """Print a summary table of all key metrics for all scenarios"""
    
    # Define scenarios and translations
    scenarios = {
        'small': 'Escenario pequeño', 
        'medium': 'Escenario mediano', 
        'full': 'Escenario grande'
    }
    
    print("# MÉTRICAS GENERALES\n")
    
    # Create a table for each scenario
    for scenario_key, scenario_name in scenarios.items():
        # Get metrics for both platforms
        jade_occupancy = load_metrics_data('JADE', 'occupancy', scenario_key)
        spade_occupancy = load_metrics_data('SPADE', 'occupancy', scenario_key)
        
        jade_capacity = load_metrics_data('JADE', 'capacity', scenario_key)
        spade_capacity = load_metrics_data('SPADE', 'capacity', scenario_key)
        
        jade_compactness = calculate_compactness_metrics('JADE', scenario_key)
        spade_compactness = calculate_compactness_metrics('SPADE', scenario_key)
        
        jade_te = load_metrics_data('JADE', 'time_eligibility', scenario_key)
        spade_te = load_metrics_data('SPADE', 'time_eligibility', scenario_key)
        
        jade_re = load_metrics_data('JADE', 'RE', scenario_key)
        spade_re = load_metrics_data('SPADE', 'RE', scenario_key)
        
        # Create DataFrame for this scenario
        data = {
            'METRICA': [
                'Ocupación / Aforo', 
                'Capacidad Exacta', 
                'Ventanas por Sala',
                'TE Value',
                'RE Value',
                'Satisfaction Score'
            ],
            'JADE': [
                format_percentage(jade_occupancy['global_stats']['tasa_ocupacion_global']) if jade_occupancy else 'N/A',
                format_percentage(jade_capacity['global_stats']['capacidad_exacta']) if jade_capacity else 'N/A',
                f"{jade_compactness['avg_windows_per_room']:.2f}" if jade_compactness else 'N/A',
                format_percentage(jade_te['te_percentage']) if jade_te else 'N/A',
                format_percentage(jade_re['re_percentage']) if jade_re else 'N/A',
                f"{jade_re.get('schedule_quality', {}).get('avg_satisfaction', 'N/A')}" if jade_re else 'N/A'
            ],
            'SPADE': [
                format_percentage(spade_occupancy['global_stats']['tasa_ocupacion_global']) if spade_occupancy else 'N/A',
                format_percentage(spade_capacity['global_stats']['capacidad_exacta']) if spade_capacity else 'N/A',
                f"{spade_compactness['avg_windows_per_room']:.2f}" if spade_compactness else 'N/A',
                format_percentage(spade_te['te_percentage']) if spade_te else 'N/A',
                format_percentage(spade_re['re_percentage']) if spade_re else 'N/A',
                f"{spade_re.get('schedule_quality', {}).get('avg_satisfaction', 'N/A')}" if spade_re else 'N/A'
            ]
        }
        df = pd.DataFrame(data)
        
        # Print scenario name and table
        print(f"## {scenario_name}:")
        display(df)
        print("\n")

def print_all_metric_tables():
    """Print all metric tables"""
    print("""# Comparative Analysis: JADE vs SPADE
    
The following tables present a comprehensive comparison of timetabling metrics between JADE and SPADE platforms across different scenarios.
    """)
    
    print_occupancy_table()
    print_capacity_table()
    print_compactness_table()
    print_te_table()
    print_re_table()
    print_general_metrics_table()

def debug_re_calculation():
    """Debug why RE metrics are identical between platforms"""
    print("# Debugging Room Eligibility (RE) Calculation\n")
    
    platforms = ['JADE', 'SPADE']
    scenarios = ['small', 'medium', 'full']
    
    for scenario in scenarios:
        print(f"## Scenario: {scenario}\n")
        
        # Compare RE values
        values = {}
        for platform in platforms:
            metrics = load_metrics_data(platform, 'RE', scenario)
            if metrics:
                values[platform] = {
                    're_metric': metrics['re_metric'],
                    're_percentage': metrics['re_percentage'],
                    'total_events': metrics['total_events'],
                    'total_rooms': metrics['total_rooms'],
                    'limited_flex_count': len(metrics.get('limited_flexibility_events', [])),
                }
        
        # Display comparison table
        if values:
            df = pd.DataFrame({
                'Metric': ['RE Value', 'RE Percentage', 'Total Events', 'Total Rooms', 'Limited Flexibility Events'],
                'JADE': [
                    values['JADE']['re_metric'],
                    values['JADE']['re_percentage'],
                    values['JADE']['total_events'],
                    values['JADE']['total_rooms'],
                    values['JADE']['limited_flex_count']
                ],
                'SPADE': [
                    values['SPADE']['re_metric'],
                    values['SPADE']['re_percentage'],
                    values['SPADE']['total_events'],
                    values['SPADE']['total_rooms'],
                    values['SPADE']['limited_flex_count']
                ],
                'Identical?': [
                    values['JADE']['re_metric'] == values['SPADE']['re_metric'],
                    values['JADE']['re_percentage'] == values['SPADE']['re_percentage'],
                    values['JADE']['total_events'] == values['SPADE']['total_events'],
                    values['JADE']['total_rooms'] == values['SPADE']['total_rooms'],
                    values['JADE']['limited_flex_count'] == values['SPADE']['limited_flex_count']
                ]
            })
            
            display(df)
            
            # Check RE calculation code to explain why values are identical
            print("### Explanation of Identical RE Values:\n")
            print("""The RE (Room Eligibility) values are identical between JADE and SPADE because the current implementation 
calculates RE based on the input data (professors and rooms) rather than the outputs (schedules).

In the current code, the `calculate_re_metric()` function only evaluates how many rooms could potentially 
accommodate each course based on constraints like capacity and campus. It does not analyze how the 
platforms actually assigned rooms in their solutions.

To properly compare the platforms' performance, the RE calculation should be modified to analyze:
1. How well each platform utilized available room options
2. How close to optimal the actual room assignments were
3. How each platform handled courses with limited room options

This would provide meaningful differences that reflect each platform's scheduling algorithm performance.""")
            
            print("\n")

# Example usage
print_occupancy_table()
print_capacity_table()
print_compactness_table()
print_te_table()
print_re_table()
print_general_metrics_table()
print_all_metric_tables()
debug_re_calculation()

## OCUPACION / AFORO

### Escenario pequeño:


,METRICA,JADE,SPADE
0,BLOQUES OCUPADOS,324,324
1,BLOQUES DESOCUPADOS,351,351
2,TASA DE OCUPACION,48.00%,48.00%




### Escenario mediano:


,METRICA,JADE,SPADE
0,BLOQUES OCUPADOS,971,972
1,BLOQUES DESOCUPADOS,829,828
2,TASA DE OCUPACION,53.94%,54.00%




### Escenario grande:


...

### Explanation of Identical RE Values:

The RE (Room Eligibility) values are identical between JADE and SPADE because the current implementation 
calculates RE based on the input data (professors and rooms) rather than the outputs (schedules).

In the current code, the `calculate_re_metric()` function only evaluates how many rooms could potentially 
accommodate each course based on constraints like capacity and campus. It does not analyze how the 
platforms actually assigned rooms in their solutions.

To properly compare the platforms' performance, the RE calculation should be modified to analyze:
1. How well each platform utilized available room options
2. How close to optimal the actual room assignments were
3. How each platform handled courses with limited room options

This would provide meaningful differences that reflect each platform's scheduling algorithm performance.


## Scenario: medium



,Metric,JADE,SPADE,Identical?
0,RE Value,0.363081,0.363081,True
1,RE Percentage,36.308140,36.308140,True
2,Total Events,166.000000,166.000000,True
3,Total Rooms,40.000000,40.000000,True
4,Limited Flexibility Events,44.000000,44.000000,True


### Explanation of Identical RE Values:

The RE (Room Eligibility) values are identical between JADE and SPADE because the current implementation 
calculates RE based on the input data (professors and rooms) rather than the outputs (schedules).

In the current code, the `calculate_re_metric()` function only evaluates how many rooms could potentially 
accommodate each course based on constraints like capacity and campus. It does not analyze how the 
platforms actually assigned rooms in their solutions.

To properly compare the platforms' performance, the RE calculation should be modified to analyze:
1. How well each platform utilized available room options
2. How close to optimal the actual room assignments were
3. How each platform handled courses with limited room options

This would provide meaningful differences that reflect each platform's scheduling algorithm performance.


## Scenario: full



,Metric,JADE,SPADE,Identical?
0,RE Value,0.435182,0.435182,True
1,RE Percentage,43.518242,43.518242,True
2,Total Events,294.000000,294.000000,True
3,Total Rooms,73.000000,73.000000,True
4,Limited Flexibility Events,37.000000,37.000000,True


### Explanation of Identical RE Values:

The RE (Room Eligibility) values are identical between JADE and SPADE because the current implementation 
calculates RE based on the input data (professors and rooms) rather than the outputs (schedules).

In the current code, the `calculate_re_metric()` function only evaluates how many rooms could potentially 
accommodate each course based on constraints like capacity and campus. It does not analyze how the 
platforms actually assigned rooms in their solutions.

To properly compare the platforms' performance, the RE calculation should be modified to analyze:
1. How well each platform utilized available room options
2. How close to optimal the actual room assignments were
3. How each platform handled courses with limited room options

This would provide meaningful differences that reflect each platform's scheduling algorithm performance.


